In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
import IPython
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Reshape,MaxPooling2D, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape

# Define the root directories
real_root_dir = '/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs'
fake_root_dirs = [
    '/kaggle/input/wavefake-test/generated_audio/ljspeech_melgan',
    '/kaggle/input/wavefake-test/generated_audio/ljspeech_melgan_large',
    '/kaggle/input/wavefake-test/generated_audio/ljspeech_waveglow'
]

def load_data():
    paths = []
    labels = []
    
    # Iterate through the real dataset
    for filename in os.listdir(real_root_dir):
        file_path = os.path.join(real_root_dir, filename)
        paths.append(file_path)
        labels.append('real')

    # Iterate through all fake directories
    for fake_root_dir in fake_root_dirs:
        for filename in os.listdir(fake_root_dir):
            file_path = os.path.join(fake_root_dir, filename)
            paths.append(file_path)
            labels.append('fake')

    print('Dataset is loaded')
    return paths, labels

paths, labels = load_data()


Dataset is loaded


In [3]:
def extract_features(fake_root_dir, real_root_dir, max_length=500):
    features = []
    labels = []
    
    for file in os.listdir(fake_root_dir):
        file_path = os.path.join(fake_root_dir, file)
        try:
            audio, _ = librosa.load(file_path, sr=16000)
            mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
            if mfccs.shape[1] < max_length:
                mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
            else:
                mfccs = mfccs[:, :max_length]
            features.append(mfccs)
            labels.append(1)  # 1 for fake
        except Exception as e:
            print(f"Error encountered while parsing file: {file_path}")
            continue
            
    for file in os.listdir(real_root_dir):
        file_path = os.path.join(real_root_dir, file)
        try:
            audio, _ = librosa.load(file_path, sr=16000)
            mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
            if mfccs.shape[1] < max_length:
                mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
            else:
                mfccs = mfccs[:, :max_length]
            features.append(mfccs)
            labels.append(0)  # 0 for real
        except Exception as e:
            print(f"Error encountered while parsing file: {file_path}")
            continue
    return np.array(features), np.array(labels)

# Example usage
x, y = extract_features(fake_root_dirs[0], real_root_dir)  # Using one fake dir for demo
print("Features shape:", x.shape)
print("Labels shape:", y.shape)


Features shape: (26200, 40, 500)
Labels shape: (26200,)


In [4]:
def generate_counterfactuals(features):
    counterfactuals = []
    for mfcc in features:
        noise = np.random.normal(0, 0.01, mfcc.shape)  # Add Gaussian noise
        counterfactuals.append(mfcc + noise)
        # You can add more transformations if desired
    return np.array(counterfactuals)


In [5]:
# Generate original training features and labels
original_features, original_labels = extract_features(fake_root_dirs[0], real_root_dir)

# Split original features into training and test sets
xtrain, xtest, ytrain, ytest = train_test_split(original_features, original_labels, test_size=0.3, random_state=42)

# Generate counterfactual samples
counterfactuals = generate_counterfactuals(xtrain)

# Combine original and counterfactual features and labels
x_combined = np.concatenate((xtrain, counterfactuals))
y_combined = np.concatenate((ytrain, ytrain))  # Same labels for counterfactuals

# Reshape the combined input to add channel dimension for the model
x_combined = x_combined.reshape((-1, 40, 500, 1))  # Reshape to match the input shape of the model


In [6]:
# Build the model
model = Sequential([
    Reshape((40, 500, 1), input_shape=x_combined.shape[1:]),  # Adjust input shape
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_combined, y_combined, epochs=20, batch_size=32, validation_data=(xtest.reshape((-1, 40, 500, 1)), ytest))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 504s 438ms/step - accuracy: 0.5519 - loss: 2.7011 - val_accuracy: 0.7716 - val_loss: 0.4898
Epoch 2/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 561s 437ms/step - accuracy: 0.8094 - loss: 0.4142 - val_accuracy: 0.8813 - val_loss: 0.2784
Epoch 3/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 497s 433ms/step - accuracy: 0.9019 - loss: 0.2332 - val_accuracy: 0.9060 - val_loss: 0.2256
Epoch 4/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 493s 430ms/step - accuracy: 0.9294 - loss: 0.1673 - val_accuracy: 0.9139 - val_loss: 0.2079
Epoch 5/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 489s 427ms/step - accuracy: 0.9498 - loss: 0.1241 - val_accuracy: 0.9097 - val_loss: 0.2475
Epoch 6/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 501s 425ms/step - accuracy: 0.9621 - loss: 0.0960 - val_accuracy: 0.9182 - val_loss: 0.2131
Epoch 7/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 498s 434ms/step - accuracy: 0.9731 - loss: 0.0712 - val_accuracy: 0.9089 - val_loss: 0.2577
Epoch 8/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 488s 425ms/step - ac

In [7]:
# Generate counterfactuals for test set
counterfactual_test = generate_counterfactuals(xtest)

# Reshape the counterfactual test data to match the input shape expected by the model
counterfactual_test = counterfactual_test.reshape((-1, 40, 500, 1))

# Get predictions for original test data
original_predictions = model.predict(xtest.reshape((-1, 40, 500, 1)))

# Get predictions for counterfactual test data
counterfactual_predictions = model.predict(counterfactual_test)

# Analyze changes in predictions
sensitivity_analysis = np.abs(original_predictions - counterfactual_predictions)

# Risk estimation can be done based on how much the predictions change
risk_threshold = 0.5  # Example threshold
risk_estimate = sensitivity_analysis > risk_threshold
print("Risk estimation based on sensitivity analysis:", risk_estimate)


246/246 ━━━━━━━━━━━━━━━━━━━━ 31s 126ms/step
246/246 ━━━━━━━━━━━━━━━━━━━━ 27s 110ms/step
Risk estimation based on sensitivity analysis: [[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
